In [1]:
%%HTML
<!DOCTYPE html>
<html>
 
<head>
  <link rel="stylesheet" type="text/css" href=".\Dashboard\override.css">
</head>
<body>
    <img src=".\Dashboard\SIMM_header.png"></img>
</body>
 
</html>

In [2]:
import os
import numpy as np
import pandas as pd
import xmltodict
import itertools
from scipy.stats import norm

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option("display.max_rows", 500, "display.max_columns", 20)

# potential output directory
out_report = r'\\icmjhbmvdropuat\Arena\Outbound\OTHER\RISK\MarketRisk\BI\artiQ\GetMRMisc'

In [3]:
with open('static/SIMM Static Data 2.4.xml', 'rb') as fd:
    simm_param = xmltodict.parse(fd.read())

simm = {}
for params in simm_param['StaticData']['SimmStaticData']:
    simm[params['ID']]=params

In [4]:
# utilities needed to load the static data and manipulate dataframes

def reset_index(df, groups):
    index_reset = list(range(len(groups)))
    return df.reset_index(level=index_reset).groupby(groups)

def filter_index(df, index_keys):
    df_index = df.index
    return df_index.droplevel([level for level in df_index.names if not level in index_keys]).tolist()

def flatten(key_prefix, val_prefix, dict_list, transform=lambda x: float(x)):
    dd = {}
    if isinstance(dict_list, list):
        for item in dict_list:
            dd.update(flatten(key_prefix, val_prefix, item, transform))
        return dd
    elif isinstance(dict_list, dict):
        key, other = None, {}
        for k, v in dict_list.items():
            if k.startswith(key_prefix):
                key = v if key is None else (key, v)
            elif k.startswith(val_prefix):
                val = v
            else:
                other.setdefault(k, flatten(key_prefix, val_prefix, v, transform))
        if other:
            return {key: other} if key else other
        else:
            return {key: transform(val)}
    else:
        return transform(dict_list)

In [5]:
# set the horizon
Horizon = 'Horizon_10d'

# class correlations
class_rho = {'rho_inter': flatten('@', '#', simm[Horizon]['RiskClassCorrelations']),
             # maps risk names to correlation buckets
             'risk_names': {'Risk_IRCurve': 'IR', 'Risk_FX': 'FX',
                            'Risk_Equity': 'Equity', 'Risk_CreditQ': 'CSR (Q)',
                            'Risk_CreditNQ': 'CSR (NQ)', 'Risk_Commodity': 'Commodity'}
             }

# credit spread risk qualifying constants
crq = {'rw': flatten('@bucket', '#', simm[Horizon]['CSRQ']['Weights']),
       'con': flatten('@', '#', simm[Horizon]['CSRQ']['ConcentrationThresholds']),
       'rho_intra': flatten('@', '#', simm[Horizon]['CSRQ']['IntraCorrelations']),
       'rho_inter': flatten('@', '#', simm[Horizon]['CSRQ']['InterCorrelations'])
       }

# credit spread risk non qualifying constants
crnq = {'rw': flatten('@bucket', '#', simm[Horizon]['CSRNQ']['Weights']),
        'con': flatten('@', '#', simm[Horizon]['CSRNQ']['ConcentrationThresholds']),
        'rho_intra': flatten('@', '#', simm[Horizon]['CSRNQ']['IntraCorrelations']),
        'rho_inter': flatten('@', '#', simm[Horizon]['CSRNQ']['InterCorrelations'])
        }

# ir constants
ir = {'rw': flatten('@', '#', simm[Horizon]['IR']['Weights']),
      'ccy': flatten('@id', '@vol', simm[Horizon]['IR']['Currencies'], transform=str),
      'rho': flatten('@', '#', simm[Horizon]['IR']['IntraCorrelations']),
      'rho_inter': float(simm[Horizon]['IR']['InterCorrelation']),
      'con': flatten('@', '#', simm[Horizon]['IR']['ConcentrationThresholds']),
      'hvr': float(simm[Horizon]['IR']['HistoricalVolatilityRatio']),
      'bucket_vol_map': flatten(
          '@crif', '@id', simm[Horizon]['IR']['Volatilities']['Weights']['Volatility'], transform=str),
      'other_map': {'Risk_Inflation': 'InflationWeight', 'Risk_XCcyBasis': 'CrossCurrencyBasisWeight'}
      }

# fx constants
fx = {'rw': flatten('@', '#', simm[Horizon]['FX']['Weights']),
      'ccy': flatten('@id', '@category', simm[Horizon]['FX']['Currencies'], transform=str),
      'con': flatten('@', '#', simm[Horizon]['FX']['ConcentrationThresholds']),
      'rho': flatten('@', '#', simm[Horizon]['FX']['Correlations']),
      
      # hardcoded by the ISDA SIMM 2.4 Spec to be High volatility currencies
      'high': {'BRL':'2','MXN':'2','TRY':'2','ZAR':'2','ARS':'3'},
      'hvr': float(simm[Horizon]['FX']['HistoricalVolatilityRatio'])
      }

# adjust the fx constants for high volatility currencies 
for cc, cat in fx['high'].items():
    fx['ccy']['Currency'][cc]=cat

# equity constants
eq = {'rw': flatten('@bucket', '#', simm[Horizon]['Equity']['Weights']),
      'con': flatten('@', '#', simm[Horizon]['Equity']['ConcentrationThresholds']),
      'rho_intra': flatten('@', '#', simm[Horizon]['Equity']['IntraCorrelations']),
      'rho_inter': flatten('@', '#', simm[Horizon]['Equity']['InterCorrelations']),
      'hvr': float(simm[Horizon]['Equity']['HistoricalVolatilityRatio'])}

# commodity constants
cm = {'rw': flatten('@bucket', '#', simm[Horizon]['Commodity']['Weights']),
      'con': flatten('@', '#', simm[Horizon]['Commodity']['ConcentrationThresholds']),
      'rho_intra': flatten('@', '#', simm[Horizon]['Commodity']['IntraCorrelations']),
      'rho_inter': flatten('@', '#', simm[Horizon]['Commodity']['InterCorrelations']),
      'hvr': float(simm[Horizon]['Commodity']['HistoricalVolatilityRatio'])}

In [6]:
# the top level aggregation keys
Agg_keys = ['PostRegulations', 'CollectRegulations', 'Counterparty']

In [7]:
def apply_correlation(var, sd, correlation_transform=lambda x: x):
    cross = 0.0
    for i, j in itertools.combinations(var.keys(), 2):
        # default correlations
        if i in sd['rho_inter']['Correlation'] and j in sd['rho_inter']['Correlation'][i]['Correlation']:
            y_bc = sd['rho_inter']['Correlation'][i]['Correlation'][j]
        elif j in sd['rho_inter']['Correlation'] and i in sd['rho_inter']['Correlation'][j]['Correlation']:
            y_bc = sd['rho_inter']['Correlation'][j]['Correlation'][i]
        cross += 2.0 * correlation_transform(y_bc) * var[i] * var[j]
    return cross

In [8]:
def calc_product(Agg_keys, M, sd):
    w = {}
    groups = Agg_keys + ['ProductClass']
    for group, df in reset_index(M, groups):
        margin = {sd['risk_names'][k]: v for k, v in df['Margin'].to_dict().items()}
        w[group] = np.sqrt((df['Margin'] * df['Margin']).sum() + apply_correlation(margin, sd))

    Simm = pd.DataFrame(w, index=['SIMM']).T
    Simm.index.set_names(groups, inplace=True)
    return Simm

In [9]:
def calc_margin_IR(K, sd):
    y_bc = sd['rho_inter']
    
    dm = {}
    for group, df in reset_index(K, Agg_keys+['ProductClass', 'RiskType']):
        S_b = ((-df['K']).clip(lower=df[['K', 'WS_ki']].min(axis=1))).to_dict()
        margin = (df['K'] * df['K']).sum()
        for i, j in itertools.combinations(S_b.keys(), 2):
            g_bc = min(i[1], j[1]) / max(i[1], j[1])
            margin += y_bc * g_bc * 2.0 * S_b[i] * S_b[j]

        dm[group[:-1] + (group[-1].replace('Vol', ''), 'Delta')] = np.sqrt(margin)

    Margin_IR = pd.DataFrame(dm, index=['Margin']).T
    Margin_IR.index.set_names(Agg_keys+['ProductClass', 'RiskType', 'MarginType'], inplace=True)
    return Margin_IR

In [10]:
def calc_margin(K, sd, bucket='WS_k', replace_vol_with=''):
    
    w = {}
    for group, full_df in reset_index(K, Agg_keys+['ProductClass', 'RiskType']):

        if 'Residual' in full_df.index:
            df = full_df.loc[full_df.index.difference(['Residual'])]
            residual = full_df.loc['Residual']['K']
        else:
            df = full_df
            residual = 0.0

        S_b = ((-df['K']).clip(lower=df[['K', bucket]].min(axis=1))).to_dict()
        group_name = group[:-1] + (group[-1].replace('Vol', replace_vol_with), 'Delta' if bucket == 'WS_k' else 'Vega')
        w[group_name] = np.sqrt((df['K'] * df['K']).sum() + apply_correlation(S_b, sd)) + residual

    Margin = pd.DataFrame(w, index=['Margin']).T
    Margin.index.set_names(Agg_keys+['ProductClass', 'RiskType', 'MarginType'], inplace=True)
    return Margin

In [11]:
def calc_curve_margin(C, sd, lamb, replace_vol_with=''):

    w = {}
    for group, full_df in reset_index(C, Agg_keys+['ProductClass', 'RiskType']): 
        margin = 0.0
        # keys to lookup lambda
        non_resi_key = group + ('NonResidual',)
        resi_key = group + ('Residual',)

        if 'Residual' in full_df.index:
            df = full_df.loc[full_df.index.difference(['Residual'])]
            residual = full_df.loc['Residual'][['K', 'CVR_k']]
        else:
            df = full_df
            residual = 0.0

        if non_resi_key in lamb:
            l = lamb.loc[non_resi_key]
            S_b = ((-df['K']).clip(lower=df[['K', 'CVR_k']].min(axis=1))).to_dict()
            cm = (df['K'] * df['K']).sum() + apply_correlation(S_b, sd, correlation_transform=lambda x: x*x)
            margin += (df['CVR_k'].sum() + l * np.sqrt(cm)).clip(min=0.0)

        if resi_key in lamb:
            l = lamb.loc[resi_key]
            margin += (residual['CVR_k'] + l * residual['K']).clip(min=0.0)

        w[group[:-1] + (group[-1].replace('Vol', replace_vol_with), 'Curvature')] = margin

    Margin = pd.DataFrame(w, index=['Margin']).T
    Margin.index.set_names(Agg_keys+['ProductClass', 'RiskType', 'MarginType'], inplace=True)
    return Margin

In [12]:
def delta_K(filtered, RiskType, sd, reporting_currency='USD', pledgor=False):
    index_name = Agg_keys + ['ProductClass', 'RiskType']
    # default empty dataframe
    delta_margin = pd.DataFrame([], columns=['Margin'], index=pd.MultiIndex(
        levels=[[]] * len(index_name), codes=[[]] * len(index_name), names=index_name))

    if RiskType == 'Risk_FX':
        groups = Agg_keys + ['ProductClass', 'RiskType', 'Qualifier']
        s_k_i = filtered.groupby(groups).agg({'Amount': 'sum', 'AmountUSD': 'sum'})
        abs_s_k_i = s_k_i.abs()

        abs_s_k_i['Concentration_bucket'] = abs_s_k_i.index.get_level_values('Qualifier').map(
            lambda ccy: sd['ccy']['Currency'].get(ccy, sd['ccy']['@defaultCategory']))
        abs_s_k_i['Concentration_threshold'] = abs_s_k_i['Concentration_bucket'].apply(
            lambda bucket: sd['con']['Delta']['Threshold'][bucket])
        abs_s_k_i['CR_b'] = np.sqrt(abs_s_k_i['AmountUSD'] / abs_s_k_i['Concentration_threshold']).clip(lower=1.0)

        # get high vol or regular risk weights
        report_vol = 'High' if reporting_currency in sd['high'] else 'Regular'
        rho = sd['rho']['Correlation']['CalculationCurrency{}Volatility'.format(report_vol)]

        s_k_i['CR_b'] = abs_s_k_i['CR_b']
        s_k_i['RW_k'] = s_k_i.index.get_level_values('Qualifier').map(
            lambda ccy: sd['rw']['Delta']['{}{}'.format(
                'High' if ccy in sd['high'] else 'Regular', report_vol
            )] if ccy != reporting_currency else 0.0)
        s_k_i['WS_k_i'] = s_k_i['RW_k'] * s_k_i['AmountUSD'] * s_k_i['CR_b']

        w = {}

        for group, df in reset_index(s_k_i, Agg_keys + ['ProductClass', 'RiskType']):
            ws_ki = df['WS_k_i'].to_dict()
            CR_k = df['CR_b'].to_dict()
            K_b = sum([x * x for x in ws_ki.values()])

            # default correlations
            for i, j in itertools.combinations(ws_ki.keys(), 2):
                rho_ij = rho['{}{}'.format(
                    'High' if i in sd['high'] else 'Regular',
                    'High' if j in sd['high'] else 'Regular',
                )]
                f_kl = min(CR_k[i], CR_k[j]) / max(CR_k[i], CR_k[j])
                K_b += 2.0 * f_kl * rho_ij * ws_ki[i] * ws_ki[j]

            w[group] = [np.sqrt(K_b), sum(ws_ki.values())]

        K = pd.DataFrame(w, index=['K', 'WS_k']).T
        K.index.set_names(Agg_keys + ['ProductClass', 'RiskType'], inplace=True)
        delta_margin = calc_margin(K, sd)

    elif RiskType == 'Risk_IRCurve':
        groups = Agg_keys + ['ProductClass', 'RiskType', 'Qualifier', 'Bucket', 'Label1', 'Label2']
        other_groups = groups[:-3]
        # need to include xccybasis swaps and inflation
        other = filtered[filtered['RiskType'].isin(['Risk_XCcyBasis', 'Risk_Inflation'])]
        # filter out just Risk_IRCurve
        filtered = filtered[filtered['RiskType'] == 'Risk_IRCurve']
        s_k_i = filtered.groupby(groups).agg({'Amount': 'sum', 'AmountUSD': 'sum'})

        o_k_i = other.groupby(other_groups).agg({'Amount': 'sum', 'AmountUSD': 'sum'})
        # first grab any inflation sensitivities
        inf = o_k_i.loc[o_k_i.index.get_level_values('RiskType') == 'Risk_Inflation']

        # get the concentration thresholds - basically ignore Label 1 and 2 (the tenor and curve index)
        sum_ski = s_k_i.groupby(other_groups).agg({'Amount': 'sum', 'AmountUSD': 'sum'})

        # add the inflation sensitivities to the sum above - and take absolute value
        abs_s_k_i = sum_ski.set_index(sum_ski.index.droplevel('RiskType')).add(
            inf.set_index(inf.index.droplevel('RiskType')), fill_value=0.0).abs()

        abs_s_k_i['Concentration_bucket'] = abs_s_k_i.index.get_level_values('Qualifier').map(
            lambda ccy: sd['ccy']['ConcentrationThresholds']['Currency'].get(
                ccy, sd['ccy']['ConcentrationThresholds']['@defaultVolatility']))
        abs_s_k_i['Concentration_threshold'] = abs_s_k_i['Concentration_bucket'].apply(
            lambda dc_bucket: sd['con']['Delta']['Threshold'][dc_bucket])
        abs_s_k_i['CR_b'] = np.sqrt(abs_s_k_i['AmountUSD'] / abs_s_k_i['Concentration_threshold']).clip(lower=1.0)

        # copy it to the weighted dataframe
        concen = abs_s_k_i['CR_b'].to_dict()
        s_k_i['CR_b'] = s_k_i.index.droplevel('RiskType').map(lambda x: concen[x[:-3]])
        s_k_i['RW_k'] = [sd['rw']['Delta']['Volatility'][sd['bucket_vol_map'][x[0]]]['Weight'][x[1].lower()]
                         for x in filter_index(s_k_i, ['Bucket', 'Label1'])]
        s_k_i['WS_k_i'] = s_k_i['RW_k'] * s_k_i['AmountUSD'] * s_k_i['CR_b']

        # set the concentration risk factor for inflation risk to what was calculated above
        # leave xccy basis swaps at 1.0
        o_k_i['CR_b'] = np.where(o_k_i.index.get_level_values('RiskType') == 'Risk_XCcyBasis',
                                 1.0, o_k_i.index.droplevel('RiskType').map(lambda x: concen[x]))
        o_k_i['RW_k'] = o_k_i.index.map(
            lambda x: sd['rw']['Delta']['Volatility'][sd['ccy']['Weights']['Currency'].get(
                x[-1], 'High')][sd['other_map'][x[-2]]])

        o_k_i['WS_k_i'] = o_k_i['RW_k'] * o_k_i['AmountUSD'] * o_k_i['CR_b']
        # group the basis + inflation factors
        other_basis = o_k_i.reset_index(
            level=[x for x in o_k_i.index.names if x != 'RiskType']).groupby(Agg_keys + ['ProductClass', 'Qualifier'])

        # temp dictionary
        w = {}

        for group, df in reset_index(s_k_i, Agg_keys + ['ProductClass', 'RiskType', 'Qualifier', 'CR_b']):
            ws_ki = df['WS_k_i'].to_dict()

            # load the xccy basis and inflation
            basis_key = group[:-3] + group[-2:-1]
            if basis_key in other_basis.groups:
                ws_ki.update(other_basis.get_group(basis_key)['WS_k_i'].to_dict())

            K_b = sum([x * x for x in ws_ki.values()])

            for i, j in itertools.combinations(ws_ki.keys(), 2):
                # default correlations
                rho_ij = 1.0
                phi_ij = 1.0

                if i == 'Risk_XCcyBasis' or j == 'Risk_XCcyBasis':
                    rho_ij = sd['rho']['CrossCurrencyBasisCorrelation']
                elif i == 'Risk_Inflation' or j == 'Risk_Inflation':
                    rho_ij = sd['rho']['InflationCorrelation']
                else:
                    phi_ij = sd['rho']['BasisRiskCorrelation'] if i[1] != j[1] else 1.0
                    if i[0] != j[0]:
                        if j[0].lower() in sd['rho']['Correlation'] and i[0].lower() in \
                                sd['rho']['Correlation'][j[0].lower()]['Correlation']:
                            rho_ij = sd['rho']['Correlation'][j[0].lower()]['Correlation'][i[0].lower()]
                        else:
                            rho_ij = sd['rho']['Correlation'][i[0].lower()]['Correlation'][j[0].lower()]

                K_b += 2.0 * phi_ij * rho_ij * ws_ki[i] * ws_ki[j]

            w[group] = [np.sqrt(K_b), sum(ws_ki.values())]

        K = pd.DataFrame(w, index=['K', 'WS_ki']).T
        K.index.set_names(Agg_keys + ['ProductClass', 'RiskType', 'Qualifier', 'CR_b'], inplace=True)
        delta_margin = calc_margin_IR(K, sd)

    elif RiskType in ['Risk_CreditNQ', 'Risk_CreditQ', 'Risk_Commodity', 'Risk_Equity']:
        groups = Agg_keys + ['ProductClass', 'RiskType', 'Bucket']

        credit = RiskType in ['Risk_CreditNQ', 'Risk_CreditQ']
        net_sensitivities_by = ['Qualifier', 'Label2'] if credit else ['Qualifier']

        # calculate the net sensitivity
        s_k_i = filtered.groupby(groups + net_sensitivities_by).agg({'Amount': 'sum', 'AmountUSD': 'sum'})

        if s_k_i.empty:
            K = pd.DataFrame([], columns=['K', 'WS_k'], index=pd.MultiIndex(
                levels=[[]] * len(groups), codes=[[]] * len(groups), names=groups))
        else:
            abs_s_k_i = s_k_i.abs()
            abs_s_k_i['Concentration_threshold'] = abs_s_k_i.index.get_level_values('Bucket').map(
                lambda bucket: sd['con']['Delta']['Threshold'][bucket])
            abs_s_k_i['CR_b'] = np.sqrt(abs_s_k_i['AmountUSD'] / abs_s_k_i['Concentration_threshold']).clip(lower=1.0)

            s_k_i['CR_b'] = abs_s_k_i['CR_b']
            s_k_i['RW_k'] = s_k_i.index.get_level_values('Bucket').map(
                lambda bucket: sd['rw']['Delta']['Weight'][bucket])
            s_k_i['WS_k_i'] = s_k_i['RW_k'] * s_k_i['AmountUSD'] * s_k_i['CR_b']

            w = {}
            for group, df in s_k_i.reset_index().groupby(groups):
                ws_ki = df['WS_k_i'].to_dict()
                CR_k = df['CR_b'].to_dict()
                q_k = df['Qualifier'].to_dict()

                K_b = sum([x * x for x in ws_ki.values()])
                for i, j in itertools.combinations(ws_ki.keys(), 2):
                    if credit:
                        issuer = 'SameName' if q_k[i][:16] == q_k[j][:16] else 'DiffName'
                        residual = 'Residual' if group[-1] == 'Residual' else 'NonResidual'
                        rho_ij = sd['rho_intra'][residual][issuer]
                    else:
                        # fetch the correlation for this bucket
                        rho_ij = sd['rho_intra']['Correlation'][group[-1]]

                    f_kl = min(CR_k[i], CR_k[j]) / max(CR_k[i], CR_k[j])
                    K_b += 2.0 * f_kl * rho_ij * ws_ki[i] * ws_ki[j]

                w[group] = [np.sqrt(K_b), sum(ws_ki.values())]

            K = pd.DataFrame(w, index=['K', 'WS_k']).T
            K.index.set_names(groups, inplace=True)
            delta_margin = calc_margin(K, sd)

    return K, delta_margin, None

In [13]:
def vega_K(filtered, RiskType, sd, reporting_currency='USD', pledgor=False):
    index_name = Agg_keys + ['ProductClass', 'RiskType']
    na = pd.DataFrame([], columns=['Margin'], index=pd.MultiIndex(
        levels=[[]] * len(index_name), codes=[[]] * len(index_name), names=index_name))
    vega_margin = na
    curve_margin = na

    alpha = norm.ppf(.99)
    filtered['t_kj'] = filtered['Label1'].apply(
        lambda x: int(x[:-1]) * ({'d': 1.0, 'w': 7.0, 'm': 365 / 12.0, 'y': 365.0}).get(x.lower()[-1]))
    # needed for curvature
    filtered['SF_t'] = 0.5 * (14 / filtered['t_kj']).clip(upper=1.0)

    if RiskType in ['Risk_IRVol', 'Risk_FXVol', 'Risk_EquityVol', 'Risk_CommodityVol']:
        irvol = RiskType == 'Risk_IRVol'
        fxvol = RiskType == 'Risk_FXVol'
        if fxvol:
            groups = Agg_keys + ['ProductClass', 'RiskType', 'Qualifier', 'Label1']
            filtered['Sigma_kj'] = filtered['Qualifier'].apply(
                lambda ccy: sd['rw']['Delta']['{}{}'.format(
                    'High' if ccy[:3] in sd['high'] else 'Regular',
                    'High' if ccy[3:] in sd['high'] else 'Regular'
                )] * np.sqrt(365.0 / 14) / alpha
            )
        elif irvol:
            groups = Agg_keys + ['ProductClass', 'RiskType', 'Bucket', 'Label1']
            # note - Explicitly assuming that the vega amount in the amountUSD column is multiplied by the atm vol
            filtered['Sigma_kj'] = 1.0
            # explicitly set the Bucket
            filtered['Bucket'] = filtered['Qualifier'].map(
                lambda ccy: sd['ccy']['ConcentrationThresholds']['Currency'].get(
                    ccy, sd['ccy']['ConcentrationThresholds']['@defaultVolatility']))
        else:
            groups = Agg_keys + ['ProductClass', 'RiskType', 'Qualifier', 'Bucket', 'Label1']
            filtered['Sigma_kj'] = filtered['Bucket'].apply(
                lambda x: sd['rw']['Delta']['Weight'].get(x) * np.sqrt(365.0 / 14) / alpha)

        filtered['Vega_Amount'] = filtered['Sigma_kj'] * filtered['Amount']
        filtered['Vega_AmountUSD'] = filtered['Sigma_kj'] * filtered['AmountUSD']

        filtered['Curve_Amount'] = filtered['SF_t'] * filtered['Vega_Amount']
        filtered['Curve_AmountUSD'] = filtered['SF_t'] * filtered['Vega_AmountUSD']

        # curvature margin for irvol is treated differently
        hvr, curve_adj = (sd['hvr'], 1.0) if not irvol else (1.0, 1.0 / (sd['hvr'] * sd['hvr']))
        VR_ik = hvr * filtered.groupby(groups).agg({'Vega_Amount': 'sum', 'Vega_AmountUSD': 'sum'})
        # note we have to flip the sign if this is the pledgor side
        CVR_ik = filtered.groupby(groups).agg(
            {'Curve_Amount': 'sum', 'Curve_AmountUSD': 'sum'})*(-1.0 if pledgor else 1.0)

        if VR_ik.empty:
            empty = [[]] * len(groups)
            K = pd.DataFrame([], columns=['K', 'WS_k'], index=pd.MultiIndex(
                levels=empty, codes=empty, names=groups))
        else:
            if not irvol:
                groups.remove('Label1') 
                
            CVR_k = CVR_ik.groupby(groups).agg({'Curve_Amount': 'sum', 'Curve_AmountUSD': 'sum'})
            VR_k = VR_ik.groupby(groups).agg({'Vega_Amount': 'sum', 'Vega_AmountUSD': 'sum'})
            abs_VR_k = VR_k.abs()

            if fxvol:
                abs_VR_k['Concentration_bucket'] = abs_VR_k.index.get_level_values('Qualifier').map(
                    lambda ccy: (sd['ccy']['Currency'].get(ccy[:3], sd['ccy']['@defaultCategory']),
                                 sd['ccy']['Currency'].get(ccy[3:], sd['ccy']['@defaultCategory']))).to_numpy()
                abs_VR_k['Concentration_threshold'] = abs_VR_k['Concentration_bucket'].apply(
                    lambda bucket: sd['con']['Vega']['Threshold'][bucket])
            else:
                abs_VR_k['Concentration_threshold'] = VR_k.index.get_level_values('Bucket').map(
                    lambda bucket: sd['con']['Vega']['Threshold'][bucket]).to_numpy()

            abs_VR_k['VCR_k'] = np.sqrt(
                abs_VR_k['Vega_AmountUSD'] / abs_VR_k['Concentration_threshold']).clip(lower=1.0)
            VR_k['VCR_k'] = abs_VR_k['VCR_k']

            if fxvol:
                VR_k['VRW'] = sd['rw']['Vega']
                group_K = Agg_keys + ['ProductClass', 'RiskType']
                CVR_k['Is_Residual'] = 'NonResidual'
            elif irvol:
                VR_k['VRW'] = sd['rw']['Vega']
                group_K = Agg_keys + ['ProductClass', 'RiskType', 'Bucket']
                CVR_k['Is_Residual'] = 'NonResidual'
            else:
                VR_k['VRW'] = VR_k.index.get_level_values('Bucket').map(
                    lambda b: sd['rw']['Vega']['Weight'].get(b, sd['rw']['Vega']['@defaultWeight']))

                group_K = Agg_keys + ['ProductClass', 'RiskType', 'Bucket']
                # filter out indices
                if RiskType == 'Risk_EquityVol':
                    CVR_k = CVR_k.index.get_level_values('Bucket').map(
                        lambda x: 1.0 if x != '12' else 0).values.reshape(-1, 1) * CVR_k

                # need to separate out residual vs non-residual
                CVR_k['Is_Residual'] = CVR_k.index.get_level_values('Bucket').map(
                    lambda x: 'Residual' if x == 'Residual' else 'NonResidual')

            VR_k['VR_k'] = VR_k['VRW'] * VR_k['Vega_AmountUSD'] * VR_k['VCR_k']

            # no concentration thresholds applied - but need to know how to calculate theta and lambda
            CVR_k['CVR_k'] = CVR_k['Curve_AmountUSD']
            CVR_k['abs_CVR_k'] = CVR_k['CVR_k'].abs()

            # group the levels by the length of the aggregation keys plus ['ProductClass', 'RiskType'] (2)
            levels = CVR_k.groupby(
                group_K[:len(Agg_keys)+2] + ['Is_Residual']).agg({'CVR_k': 'sum', 'abs_CVR_k': 'sum'})
            
            theta = (levels['CVR_k'] / levels['abs_CVR_k']).clip(upper=0.0)
            lamb = (norm.ppf(0.995) ** 2 - 1.0) * (1 + theta) - theta

            # first calc Vega
            w = {}
            for group, df in VR_k.reset_index().groupby(group_K):
                # check if this is not ir vol
                if not irvol:
                    rho_ij = sd['rho']['CorrelationVol'] if fxvol else sd['rho_intra']['Correlation'][group[-1]]
                    vr_k = df['VR_k'].to_dict()
                    VCR_k = df['VCR_k'].to_dict()
                else:
                    vr_k = df[['Label1', 'VR_k']].set_index('Label1').to_dict()['VR_k']

                K_b = sum([x * x for x in vr_k.values()])

                for i, j in itertools.combinations(vr_k.keys(), 2):
                    # default correlations
                    if irvol:
                        rho_ij = sd['rho']['Correlation'][i.lower()]['Correlation'][j.lower()] \
                            if i.lower() in sd['rho']['Correlation'] else sd[
                            'rho']['Correlation'][j.lower()]['Correlation'][i.lower()]

                    f_kl = 1.0 if irvol else min(VCR_k[i], VCR_k[j]) / max(VCR_k[i], VCR_k[j])
                    K_b += 2.0 * f_kl * rho_ij * vr_k[i] * vr_k[j]

                w[group] = [np.sqrt(K_b), sum(vr_k.values())]

            # now curvature
            c = {}
            for group, df in CVR_k.reset_index().groupby(group_K):
                if not irvol:
                    cvr_k = df['CVR_k'].to_dict()
                else:
                    cvr_k = df[['Label1', 'CVR_k']].set_index('Label1').to_dict()['CVR_k']

                K_b = sum([x * x for x in cvr_k.values()])
                if not irvol:
                    rho2_ij = (sd['rho']['CorrelationVol'] if fxvol else sd['rho_intra']['Correlation'][group[-1]]) ** 2

                for i, j in itertools.combinations(cvr_k.keys(), 2):
                    if irvol:
                        rho2_ij = (sd['rho']['Correlation'][i.lower()]['Correlation'][j.lower()]
                                   if i.lower() in sd['rho']['Correlation'] else
                                   sd['rho']['Correlation'][j.lower()]['Correlation'][i.lower()]) ** 2

                    # note rho2_ij is rho_ij squared
                    K_b += 2.0 * rho2_ij * cvr_k[i] * cvr_k[j]

                c[group] = [np.sqrt(K_b), sum(cvr_k.values())]

            K = pd.DataFrame(w, index=['K', 'VR_k']).T
            K.index.set_names(group_K, inplace=True)
            vega_margin = calc_margin(K, sd, bucket='VR_k', replace_vol_with='Curve' if irvol else '')

            C = pd.DataFrame(c, index=['K', 'CVR_k']).T
            C.index.set_names(group_K, inplace=True)
            curve_margin = curve_adj * calc_curve_margin(C, sd, lamb, replace_vol_with='Curve' if irvol else '')
    else:
        if not filtered.empty:
            print('todo!')
        groups = Agg_keys + ['ProductClass', 'RiskType', 'Qualifier', 'Bucket', 'TradeID']
        empty = [[]] * len(groups)
        K = pd.DataFrame([], columns=['K', 'WS_k'], index=pd.MultiIndex(
            levels=empty, codes=empty, names=groups))
        # inflation/credit - TODO!!

    return K, vega_margin, curve_margin

In [14]:
# here we load a crif file
import glob

# crif = pd.read_csv('\\\\ICMJHBMVDROPUAT\\Arena\\Outbound\\OTHER\\Collateral\\Simm\\CRIF_2021-06-24.tsv', sep='\t', dtype={'Bucket': str} )
agreementLookup = {'Goldman Sachs Int'               : 'GSIL_TM',
                   'Goldman Sachs International ZAR' : 'GSIL_TM',
                   'GS Bank'                         : 'GA_Bank_TM',
                   'Citibank NA NY'                  : 'Citibank NA_TM',
                   'Citigroup G Mkt Ldn'             : 'IB CGMI',
                   'Citigroup Global Markets'        : 'CitiGroup_TM',
                   'Citibank Jhb'                    : 'Citibank NA_TM',
                   'Citibank London'                 : 'Citibank NA_TM',  
                   'Citibank NA NY'                  : 'Citibank NA_TM',
                   'Soc Gen Jhb'                     : 'SocGen_IMTM',
                   'Soc Gen Ldn'                     : 'SocGen_IMTM',
                   'Soc Gen Paris'                   : 'SocGen_IMTM',
                   'UBS AG London'                   : 'UBS_IMTM',
                   'UBS AG Zurich'                   : 'UBS_IMTM'}

#agreementLookup = {'Goldman Sachs Int'               : 'GSIL_TM',
#                   'Goldman Sachs International ZAR' : 'GSIL_TM'}


In [15]:
riskgroups = {
    'Risk_Equity': (delta_K, eq),
    'Risk_Commodity': (delta_K, cm),
    'Risk_FX': (delta_K, fx),
    'Risk_CreditQ': (delta_K, crq),
    'Risk_IRCurve': (delta_K, ir),
    'Risk_IRVol': (vega_K, ir),
    'Risk_EquityVol': (vega_K, eq),
    'Risk_FXVol': (vega_K, fx),
    'Risk_CommodityVol': (vega_K, cm)
}


def report(total, Agg_keys):
    IM_x = total.groupby(Agg_keys + ['ProductClass', 'RiskType'])['Margin'].sum()
    SIMM_p = calc_product(Agg_keys, IM_x, class_rho)
    SIMM = SIMM_p.groupby(Agg_keys).sum('SIMM')

    total['Exposure'] = IM_x
    total['ProductTotal'] = SIMM_p
    total['SIMM_Placed'] = SIMM

    total.set_index('Exposure', append=True, inplace=True)
    total.set_index('ProductTotal', append=True, inplace=True)
    total.set_index('SIMM_Placed', append=True, inplace=True)

    return total.reorder_levels(
       Agg_keys + ['SIMM_Placed', 'ProductClass', 'ProductTotal', 'RiskType', 'Exposure', 'MarginType'])


def calc_simm(crif):
    # set the correct groups for the different riskTypes
    crif['RiskGroup'] = crif['RiskType'].apply(
        lambda x: 'Risk_IRCurve' if x in ['Risk_XCcyBasis', 'Risk_Inflation'] else x)
    
    index_names = Agg_keys + ['ProductClass', 'RiskType']
    total_pledgor = pd.DataFrame([], columns=['Margin'], index=pd.MultiIndex(
        levels=[[]] * len(index_names), codes=[[]] * len(index_names), names=index_names))
    total_secured = pd.DataFrame([], columns=['Margin'], index=pd.MultiIndex(
        levels=[[]] * len(index_names), codes=[[]] * len(index_names), names=index_names))
    
    for group, df in crif.groupby(['PostRegulations', 'CollectRegulations', 'ProductClass', 'RiskGroup']):
        riskgroup = group[-1]
        margin_calc, params = riskgroups[riskgroup]
        K, m1, m2 = margin_calc(df.copy(), riskgroup, params, pledgor=False)

        # m1 is either delta or vega - same for pledgor and secured
        total_pledgor = total_pledgor.add(m1, fill_value=0.0)
        total_secured = total_secured.add(m1, fill_value=0.0)

        if margin_calc == vega_K:
            # m2 is set at secured (pledgor=False)
            total_secured = total_secured.add(m2, fill_value=0.0)
            K, m1, m2 = margin_calc(df, riskgroup, params, pledgor=True)
            # m2 is now pledgor
            total_pledgor = total_pledgor.add(m2, fill_value=0.0)

    secured = report(total_secured.droplevel('PostRegulations'), ['CollectRegulations', 'Counterparty'])
    pledgor = report(total_pledgor.droplevel('CollectRegulations'), ['PostRegulations', 'Counterparty'])

    return {'Pledgor':pledgor,'Secured':secured}
    

In [16]:
# ipywidgets (used to draw the dashboard) github (also with a link to docs) https://github.com/jupyter-widgets/ipywidgets
import ipywidgets as widgets
from IPython.display import display, HTML


In [17]:
class SIMM(object):
    def __init__(self, crif_paths, cache):
        self.cache    = cache
        self.paths    = crif_paths
        self.source   = widgets.Dropdown(
            options=['Arcaida CRIF', 'Adaptiv CRIF'], 
            value='Adaptiv CRIF', description='CRIF Source:', disabled=False)
        
        self.load_dates(self.paths[self.source.value])
        
        self.active_date = widgets.Select(
            options=self.dates, value=self.dates[-1], rows=5, 
            description='RunDate:', disabled=False, layout = {'width': 'max-content'})
        
        self.lod         = widgets.RadioButtons(
            options=['SIMM', 'ByProduct', 'ByExposure', 'ByMargin'],
            layout={'width': 'max-content'},
            description='Detail:', disabled=False)        
        
        self.status = widgets.ToggleButtons(
            options = ['Pledgor','Secured'],
            description='Simm Type:',
            disabled=False,
            tooltips=['Description of Pledgor', 'Description of Secured'])
        
        self.output = widgets.Output()
        self.label  = widgets.Label(self.active_date.value)
        self.load_crif(self.active_date.value)        
        
        self.source.observe(self.source_changed, 'value')
        self.status.observe(self.status_changed, 'value')
        self.active_date.observe(self.crif_changed, 'value')
        self.lod.observe(self.lod_changed, 'value')    
        # the results
        self.panel = widgets.VBox([self.source, self.active_date, self.status, self.lod, self.label, self.output])
        
    def load_dates(self, path, history=10):
        self.alldates    = sorted(glob.glob(path))[-history:]
        self.date_lookup = {os.path.split(x)[-1]:x for x in self.alldates}
        self.dates       = list(self.date_lookup.keys())
        
    def load_results(self, lod):
        if lod == 'SIMM':
            df = self.report.reset_index().iloc[:,:3].drop_duplicates().set_index(self.report.index.names[:1])
        elif lod == 'ByProduct':
            df = self.report.reset_index().iloc[:,:5].drop_duplicates().set_index(self.report.index.names[:4])
        elif lod == 'ByExposure':
            df = self.report.reset_index().iloc[:,:7].drop_duplicates().set_index(self.report.index.names[:6])
        else:
            df = self.report
        # save this file
        filename = './files/{}_{}_{}.csv'.format(self.label.value.replace('.tsv',''), lod, self.status.value)
        if not os.path.isfile( filename ):
            df.to_csv(filename)
            
        download = HTML(
            '<a href="https://icmjhbmvaisprd3:8888/tree/{}" title="click to download">Download</a>'.format(filename))
        
        self.output.clear_output(wait=True)
        with self.output:        
            display(df)
            display(download)

    def load_crif(self, date):
        if date not in self.cache:
            crif = pd.read_csv(self.date_lookup[date], sep='\t', dtype={'Bucket': str})
            if self.source.value.startswith('Arcaida'):
                crif['Counterparty'] = crif['PortfolioID']
                crif['IMModel'] = crif['ImModel']
            else:
                crif.loc[:,'Counterparty'] = crif['Counterparty'].apply(lambda x: agreementLookup.get(x,x))
            self.cache[date] = calc_simm(crif)
            
        self.report = self.cache[date][self.status.value]
        self.label.value = date
        self.load_results(self.lod.value)        
        
    def lod_changed(self, x):
        self.load_results(x['new'])
            
    def status_changed(self, x):
        self.report = self.cache[self.active_date.value][self.status.value]
        self.load_results(self.lod.value)
            
    def source_changed(self, x):
        self.load_dates(self.paths[x['new']])
        self.active_date.options = self.dates
        self.active_date.value = self.dates[-1]
            
    def crif_changed(self, x):        
        self.load_crif(x['new'])
        

In [19]:
cache = {}
simm_left = SIMM(
    {'Adaptiv CRIF':'\\\\ICMJHBMVDROPPRD\\Arena\\Outbound\\OTHER\\Collateral\\Simm\\CRIF_20*.tsv', 
     'Arcaida CRIF':'\\\\ICMJHBMVDROPPRD\\Arena\\Outbound\\OTHER\\Collateral\\Simm\\CRIF_AcadiaSoft_*.tsv'}, cache)
simm_right = SIMM(
    {'Adaptiv CRIF':'\\\\ICMJHBMVDROPPRD\\Arena\\Outbound\\OTHER\\Collateral\\Simm\\CRIF_20*.tsv', 
     'Arcaida CRIF':'\\\\ICMJHBMVDROPPRD\\Arena\\Outbound\\OTHER\\Collateral\\Simm\\CRIF_AcadiaSoft_*.tsv'}, cache)

# display GUI for Voila
display(widgets.HBox([simm_left.panel, simm_right.panel]))